<h1 align=center><font size = 6>Segmenting and Clustering Neighborhoods in Philadelphia</font></h1>

## 1. Get, Clean and Transform data into Dataframe

#### Install packages

In [1]:
!pip install lxml
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install beautifulsoup4
!pip install uszipcode
!pip install geocoder

     |████████████████████████████████| 5.8MB 11.4MB/s eta 0:00:01
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##

#### Import libraries

In [2]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


import folium # map rendering library


from bs4 import BeautifulSoup

import geocoder # import geocoder
from uszipcode import SearchEngine

### Scrape neighborhoods data from Wiki and transform to dataframe

In [9]:
source = requests.get('https://en.wikipedia.org/wiki/Callowhill,_Philadelphia').text
soup = BeautifulSoup(source, 'html5lib')
all_li_tag= []
for x in soup.find_all('li'):
    all_li_tag.append(x.text)
indexes = range(0,14)
for index in sorted(indexes, reverse=True):
    del all_li_tag[index]
philly_neighborhoods = pd.DataFrame({"Borough":'',"Neighborhoods":all_li_tag})
philly_neighborhoods = philly_neighborhoods.drop(philly_neighborhoods.tail(79).index)
philly_neighborhoods

,Borough,Neighborhoods
0,,Avenue of the Arts
1,,Broad Street
2,,Chinatown
3,,Fitler Square
4,,Franklin Square
5,,Jewelers' Row
6,,Logan Square
7,,Old City
8,,Penn's Landing
9,,Rittenhouse Square


In [10]:
##Add Borough to df1
philly_neighborhoods.loc[0:12]['Borough'] = "Center City"
philly_neighborhoods.loc[13:48]['Borough'] = "South"
philly_neighborhoods.loc[49:59]['Borough'] = "Southwest"
philly_neighborhoods.loc[60:111]['Borough'] = "Lower North"
philly_neighborhoods.loc[112:120]['Borough'] = "North, Uper North"
philly_neighborhoods.loc[121:128]['Borough'] = "Olney-Oak Lane"
philly_neighborhoods.loc[129:134]['Borough'] = "Lower Northwest"
philly_neighborhoods.loc[135:141]['Borough'] = "Uper Northwest"
philly_neighborhoods.loc[142:157]['Borough'] = "Near Northeast"
philly_neighborhoods.loc[158:172]['Borough'] = "Far Northeast"
philly_neighborhoods.loc[173:]['Borough'] = "River Wards"

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

### Get coordintates for neighborhoods - Add them to dataframe

In [ ]:
philly_neighborhoods["lon"] = np.nan  #tab to show suggestion
philly_neighborhoods["lat"] = np.nan

geolocator = Nominatim(user_agent="philly_explorer")
for i in range(len(philly_neighborhoods)):
    #print(i)
    address = philly_neighborhoods['Neighborhoods'][i] + ", Philadelphia, Pennsylvania" # df1[i][2]
    location = None
    for j in range(3):
        location = geolocator.geocode(address)
        if location is not None:
            break

    if location is None:
        continue
    philly_neighborhoods["lat"][i] = location.latitude    
    philly_neighborhoods["lon"][i] = location.longitude
   
    #print(address)

In [3]:
#save dataframe to csv file since geolocator is not consistant and cannot run too many times
philly_neighborhoods.to_csv('philly_coordinates.csv')

NameError: name 'philly_neighborhoods' is not defined

#### Open dataframe from CSV, dataframe now includes: Borough, Neighborhoods, Latitude(lat) and Longtitude(lon)

In [5]:
philly_data = pd.read_csv("philly_coordinates.csv") 
philly_data = philly_data[["Borough","Neighborhoods","lat","lon"]]
philly_data.dropna(inplace = True)
philly_data.head()

,Borough,Neighborhoods,lat,lon
1,Center City,Broad Street,39.914203,-75.171521
2,Center City,Chinatown,39.953446,-75.154622
3,Center City,Fitler Square,39.947603,-75.179784
4,Center City,Franklin Square,39.955383,-75.150363
6,Center City,Logan Square,39.958333,-75.171125


### Use uszipcode library to get population density and average income for each neighborhood

#### Get population_density, average household income for each neighborhood. Add them to dataframe

In [6]:
search = SearchEngine(simple_zipcode=True) # set simple_zipcode=False to use rich info database

# philly_data['average income'] = np.nan
average_income = []
population = []
average_housing_units = []
average_home_value = []
for x in range(len(philly_data)):
    zipcode = search.by_coordinates(philly_data['lat'].iloc[x], philly_data['lon'].iloc[x], radius=20)
    list_0=[]
    list_1 = []
    list_2 = []
    list_3 = []
    for i in range(len(zipcode)):
        if abs(zipcode[i].bounds_south) <abs(philly_data['lat'].iloc[x])<abs(zipcode[i].bounds_north) and abs(zipcode[i].bounds_east) <abs(philly_data['lon'].iloc[x])<abs(zipcode[i].bounds_west):
            list_0.append(zipcode[i].median_household_income)
            list_1.append(zipcode[i].population_density)
            list_2.append(zipcode[i].housing_units)
            list_3.append(zipcode[i].median_home_value)
            #print(list_0)
            list_0 = [q for q in filter(None, list_0)]   ## make a list and remove the Nove value
            list_1 = [k for k in filter(None, list_1)] 
            list_2 = [g for g in filter(None, list_2)] 
            list_3 = [t for t in filter(None, list_3)] 
    avg = np.mean(list_0) #calculating mean
    avg_pop = np.mean(list_1)
    avg_housing_units = np.mean(list_2)
    avg_home_value = np.mean(list_3)
    
    #print(avg)
    #print(x)
    
    average_income.append(avg) #add values to a list
    population.append(avg_pop)
    average_housing_units.append(avg_housing_units)
    average_home_value.append(avg_home_value)
    
average_income = np.array(average_income) #transform list to numpy array
population = np.array(population)
average_housing_units = np.array(average_housing_units)
average_home_value = np.array(average_home_value)

philly_data['average income'] =average_income
philly_data['population density'] = population
philly_data['average housing units'] = average_housing_units
philly_data['average home value'] = average_home_value

Start downloading data for simple zipcode database, total size 9MB ...
  1 MB finished ...
  2 MB finished ...
  3 MB finished ...
  4 MB finished ...
  5 MB finished ...
  6 MB finished ...
  7 MB finished ...
  8 MB finished ...
  9 MB finished ...
  10 MB finished ...
  Complete!


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [7]:
## final dataframe
philly_data.head()

,Borough,Neighborhoods,lat,lon,average income,population density,average housing units,average home value
1,Center City,Broad Street,39.914203,-75.171521,37587.0,10744.0,21220.5,158150.0
2,Center City,Chinatown,39.953446,-75.154622,69061.0,20806.0,7988.0,379800.0
3,Center City,Fitler Square,39.947603,-75.179784,52888.5,27533.0,17835.0,310700.0
4,Center City,Franklin Square,39.955383,-75.150363,93720.0,14527.0,7576.0,453400.0
6,Center City,Logan Square,39.958333,-75.171125,63215.0,26748.5,15486.0,384250.0


### Create a map of Philly Neighborhoods

In [8]:
!jupyter notebook --ip=0.0.0.0 --allow-root #uncomment this if jupyter fail to get addresss from geolocator package

The Jupyter HTML Notebook.

This launches a Tornado based HTML Notebook Server that serves up an
HTML5/Javascript Notebook client.

Subcommands
-----------

Subcommands are launched as `jupyter-notebook cmd [args]`. For information on
using subcommand 'cmd', do: `jupyter-notebook cmd -h`.

list
    List currently running notebook servers.
stop
    Stop currently running notebook server for a given port
password
    Set a password for the notebook server.

Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--debug
    set log level to logging.DEBUG (maximize logging output)
--generate-config
    generate default config file
-y
    Answer yes to any questions instead of prompting.
--no-browser
    Don't open the notebook in a browser after startup.
--pylab
    DISABLED: use %pylab or %matplotlib in the notebook to enable matplotlib.
--

In [9]:
geolocator = Nominatim(user_agent="phil_explorer")
location = geolocator.geocode("Philadelphia, PA")
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Philadelphia are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Philadelphia are 39.9527237, -75.1635262.


In [10]:
map_philly = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(philly_data['lat'], philly_data['lon'], philly_data['Neighborhoods']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_philly)  
    
map_philly

### Define Fousquare Credentials and Version

In [11]:
CLIENT_ID = 'PA5ZIKIU0LO3MTUN3U42ZN4Y1J3LN4UGYNUOJNZDBZ4GHE4Y' # your Foursquare ID
CLIENT_SECRET = 'ORMC5BSDWVJ4PUC0FKKLNPMNVZL50BJOQBV5SQILHQBYUTTL' # your Foursquare Secret
VERSION = '20191012' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

radius = 500
LIMIT = 100

Your credentails:
CLIENT_ID: PA5ZIKIU0LO3MTUN3U42ZN4Y1J3LN4UGYNUOJNZDBZ4GHE4Y
CLIENT_SECRET:ORMC5BSDWVJ4PUC0FKKLNPMNVZL50BJOQBV5SQILHQBYUTTL


### Create a function to explore all neighborhoods in Philadelphia

In [12]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhoods', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Get revenues in Philadelphia neighbourhoods

In [16]:

philly_venues = getNearbyVenues(names=philly_data['Neighborhoods'],
                                   latitudes=philly_data['lat'],
                                   longitudes=philly_data['lon']
                                  )


Broad Street
Chinatown
Fitler Square
Franklin Square
Logan Square
Old City
Penn's Landing
Rittenhouse Square
Society Hill
South Street
Washington Square West
Bella Vista
Central South Philadelphia
Devil's Pocket
Dickinson Square West
FDR Park
Graduate Hospital
Grays Ferry
Greenwich
Hawthorne
Industrial
Italian Market
Lower Moyamensing
Marconi Plaza
Moyamensing
Packer Park
Passyunk Square
Pennsport
Point Breeze
Queen Village
Schuylkill
Southwark
Southwest Center City
South Philadelphia/East
South Philadelphia/West
Sports Complex
Tasker
West Passyunk
Wharton
Whitman
Wilson Park
Angora
Bartram Village
Clearview
Eastwick
Elmwood Park
Hog Island
Kingsessing
Paschall
Penrose
Southwest Schuylkill
Belmont Village
Carroll Park
Cathedral Park
Centennial District
Cedar Park
Cobbs Creek
Dunlap
Garden Court
Haddington
Haverford North
Mantua
Mill Creek
Overbrook
Overbrook Park
Parkside
Powelton Village
Saunders Park
Spruce Hill
30th Street Station
University City
Walnut Hill
Woodland Terrace
Wynnefi

In [17]:
##check size of philly_venues df
print(philly_venues.shape)
philly_venues.head()

(4047, 7)


,Neighborhoods,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Broad Street,39.914203,-75.171521,Marconi Plaza,39.915290,-75.171812,Park
1,Broad Street,39.914203,-75.171521,Chickie's & Pete's,39.911639,-75.174636,Sports Bar
2,Broad Street,39.914203,-75.171521,Termini Bros,39.911409,-75.175342,Bakery
3,Broad Street,39.914203,-75.171521,Pastificio,39.911404,-75.175002,Deli / Bodega
4,Broad Street,39.914203,-75.171521,Pop's Homemade Water Ice,39.916995,-75.168143,Ice Cream Shop


In [18]:
#how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(philly_venues['Venue Category'].unique())))

There are 311 uniques categories.


## 2. Analyze Each Neighborhood

In [19]:
# one hot encoding
philly_onehot = pd.get_dummies(philly_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
philly_onehot['Neighborhoods'] = philly_venues['Neighborhoods'] 

# move neighborhood column to the first column
fixed_columns = [philly_onehot.columns[-1]] + list(philly_onehot.columns[:-1])
toronto_onehot = philly_onehot[fixed_columns]

philly_onehot.head()

,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Betting Shop,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Basketball Court,College Bookstore,College Gym,College Hockey Rink,College Quad,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dutch Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hockey Rink,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lebanese Restaurant,Light Rail Station,Lighting Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roller Rink,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stadium,Steakhouse,Storage Facility,Strip Club,Student Center,Supermarket,Supplement Shop,Sushi Restaurant,

In [20]:
philly_onehot.shape

(4047, 312)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [21]:

philly_grouped = philly_onehot.groupby('Neighborhoods').mean().reset_index()
philly_grouped

,Neighborhoods,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Betting Shop,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Basketball Court,College Bookstore,College Gym,College Hockey Rink,College Quad,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dutch Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hockey Rink,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lebanese Restaurant,Light Rail Station,Lighting Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roller Rink,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stadium,Steakhouse,Storage Facility,Strip Club,Student Center,Supermarket,Supplement Shop,Sus

In [22]:
philly_grouped.shape

(157, 312)

#### Print each neighborhood along with the top 5 most common venues

In [23]:
num_top_venues = 5

for hood in philly_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = philly_grouped[philly_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----30th Street Station----
                 venue  freq
0  Rental Car Location  0.13
1       Sandwich Place  0.10
2           Food Truck  0.10
3  American Restaurant  0.07
4                  Pub  0.07


----Academy Gardens----
                 venue  freq
0               Garden   0.5
1           Donut Shop   0.5
2       Adult Boutique   0.0
3    Other Repair Shop   0.0
4  Peruvian Restaurant   0.0


----Allegheny West----
                  venue  freq
0          Intersection  0.17
1  Fast Food Restaurant  0.10
2        Sandwich Place  0.07
3        Shopping Plaza  0.07
4                   Bar  0.03


----Andorra----
                   venue  freq
0             Playground   1.0
1         Adult Boutique   0.0
2        Other Nightlife   0.0
3  Performing Arts Venue   0.0
4                   Park   0.0


----Angora----
                 venue  freq
0   Chinese Restaurant  0.18
1       Cosmetics Shop  0.09
2  American Restaurant  0.09
3                 Road  0.09
4                 Park  0.0


#### Put results into a pandas dataframe

Write a function to sort the venues in descending order.


In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood.

In [52]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = philly_grouped['Neighborhoods']

for ind in np.arange(philly_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(philly_grouped.iloc[ind, :], num_top_venues) # loop through range horizontally for each row 

neighborhoods_venues_sorted.head()

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,30th Street Station,Rental Car Location,Food Truck,Sandwich Place,Pub,American Restaurant,Café,Trail,Train Station,Southern / Soul Food Restaurant,Fast Food Restaurant
1,Academy Gardens,Garden,Donut Shop,Zoo Exhibit,Field,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant
2,Allegheny West,Intersection,Fast Food Restaurant,Shopping Plaza,Sandwich Place,Train Station,Supermarket,Shoe Store,Chinese Restaurant,Grocery Store,Liquor Store
3,Andorra,Playground,Zoo Exhibit,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant
4,Angora,Chinese Restaurant,Cosmetics Shop,Supermarket,Park,Train Station,Discount Store,Market,Grocery Store,American Restaurant,Road


## 4. Cluster Neighborhoods

#### Run *k*-means to cluster the neighborhood into 10 clusters.

In [53]:
# set number of clusters
kclusters = 10

philly_grouped_clustering = philly_grouped.drop('Neighborhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(philly_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([2, 3, 2, 2, 4, 3, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 4, 2, 2, 2,
       4, 2, 2, 2, 2, 2, 2, 4, 2, 1, 2, 1, 2, 2, 0, 2, 4, 4, 2, 2, 2, 2,
       2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4,
       2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 4, 2, 2,
       2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 2, 1, 2, 2, 2, 2, 1, 4, 2,
       2, 4, 2], dtype=int32)

#### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [54]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

philly_merged = philly_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
philly_merged = philly_merged.join(neighborhoods_venues_sorted.set_index('Neighborhoods'), on='Neighborhoods')

philly_merged.head() # check the last columns!

,Borough,Neighborhoods,lat,lon,average income,population density,average housing units,average home value,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Center City,Broad Street,39.914203,-75.171521,37587.0,10744.0,21220.5,158150.0,2.0,Bar,Pizza Place,Sports Bar,Deli / Bodega,Ice Cream Shop,Sandwich Place,Pharmacy,Sushi Restaurant,Park,Bank
2,Center City,Chinatown,39.953446,-75.154622,69061.0,20806.0,7988.0,379800.0,2.0,Chinese Restaurant,Bakery,Deli / Bodega,Ice Cream Shop,Vietnamese Restaurant,Shanghai Restaurant,Breakfast Spot,Italian Restaurant,Bubble Tea Shop,Tea Room
3,Center City,Fitler Square,39.947603,-75.179784,52888.5,27533.0,17835.0,310700.0,2.0,Café,Coffee Shop,Italian Restaurant,Trail,Food Truck,Pizza Place,Breakfast Spot,Gastropub,Grocery Store,Park
4,Center City,Franklin Square,39.955383,-75.150363,93720.0,14527.0,7576.0,453400.0,2.0,Chinese Restaurant,Tea Room,History Museum,Bar,Asian Restaurant,Music Venue,Creperie,Food Truck,Café,Park
6,Center City,Logan Square,39.958333,-75.171125,63215.0,26748.5,15486.0,384250.0,2.0,Coffee Shop,Pizza Place,Hotel,Convenience Store,Bakery,Park,Science Museum,Art Museum,Outdoor Sculpture,Pet Store


In [55]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(philly_merged['lat'], philly_merged['lon'], philly_merged['Neighborhoods'], philly_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

TypeError: list indices must be integers or slices, not float

## 3. Examine Clusters

In [ ]:
cluster_1 = philly_merged.loc[philly_merged['Cluster Labels'] == 0, philly_merged.columns[[1] + list(range(4, philly_merged.shape[1]))]]
cluster_1.sort_values(by=['population density','average home value'], ascending=False).head()

In [ ]:
cluster_2 = philly_merged.loc[philly_merged['Cluster Labels'] == 1, philly_merged.columns[[1] + list(range(4, philly_merged.shape[1]))]]
cluster_2.sort_values(by=['population density','average home value'], ascending=False).head()

In [ ]:
cluster_3 = philly_merged.loc[philly_merged['Cluster Labels'] == 2, philly_merged.columns[[1] + list(range(4, philly_merged.shape[1]))]]
cluster_3.sort_values(by=['population density','average income'], ascending=False).head()

In [ ]:
cluster_4 = philly_merged.loc[philly_merged['Cluster Labels'] == 3, philly_merged.columns[[1] + list(range(4, philly_merged.shape[1]))]]
cluster_4.sort_values(by=['population density','average income'], ascending=False).head()

In [ ]:
cluster_5 = philly_merged.loc[philly_merged['Cluster Labels'] == 4, philly_merged.columns[[1] + list(range(4, philly_merged.shape[1]))]]
cluster_5.sort_values(by=['population density','average income'], ascending=False).head()

In [ ]:
cluster_6 = philly_merged.loc[philly_merged['Cluster Labels'] == 5, philly_merged.columns[[1] + list(range(4, philly_merged.shape[1]))]]
cluster_6.sort_values(by=['population density','average income'], ascending=False).head()

In [ ]:
cluster_7 = philly_merged.loc[philly_merged['Cluster Labels'] == 6, philly_merged.columns[[1] + list(range(4, philly_merged.shape[1]))]]
cluster_7.sort_values(by=['population density','average home value'], ascending=False).head()

In [ ]:
cluster_8 = philly_merged.loc[philly_merged['Cluster Labels'] == 7, philly_merged.columns[[1] + list(range(4, philly_merged.shape[1]))]]
cluster_8.sort_values(by=['population density','average income'], ascending=False).head()

In [ ]:
cluster_9 = philly_merged.loc[philly_merged['Cluster Labels'] == 8, philly_merged.columns[[1] + list(range(4, philly_merged.shape[1]))]]
cluster_9.sort_values(by=['population density' ,'average income'], ascending=False).head()

In [ ]:
cluster_10 = philly_merged.loc[philly_merged['Cluster Labels'] == 9, philly_merged.columns[[1] + list(range(4, philly_merged.shape[1]))]]
cluster_10.sort_values(by=['population density','average income'], ascending=False).head()